In [ ]:
from huggingface_hub import notebook_login
notebook_login()


Python 3.12

In [ ]:
!pip install -q torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0
!pip install -q transformers==4.44.0 datasets==2.19.0 accelerate==0.34.0 trl==0.9.6 loralib evaluate==0.4.2 scipy==1.11.4 numpy==1.26.4 sentencepiece protobuf==4.25.3
!pip install -q numpy==1.26.4 scipy==1.11.4
# restrart runtime after this command from Runtime tab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load model without device_map="auto" initially to ensure all weights are materialized on CPU.
# This prevents layers from being on a 'meta' device, allowing loralib to inject properly.
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    # Removed device_map="auto" here
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# Using "eager" attention implementation as recommended by earlier stderr for Gemma2 models
model.config.attn_implementation = "eager"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # This check is now less likely to trigger if model is fully loaded to CPU
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
_inject_lora(model)

# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))


# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# Manually move the model to GPU *after* LoRA injection.
# This ensures the model is on the correct device before Trainer takes over.
if torch.cuda.is_available():
    model.to("cuda")

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%)")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)
x

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()


os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model (no quantization)
# --------------------
# --------------------
# Load tokenizer & base model (4-bit quantized)
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    device_map="auto",
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

model.config.attn_implementation = "flash_attention_2"


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # Skip meta tensors (lazy-loaded weights that aren't materialized yet)
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
# actually inject
_inject_lora(model)

# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))



# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%)")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)


In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load model without device_map="auto" initially to ensure all weights are materialized on CPU.
# This prevents layers from being on a 'meta' device, allowing loralib to inject properly.
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    # Removed device_map="auto" here
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# Using "eager" attention implementation as recommended by earlier stderr for Gemma2 models
model.config.attn_implementation = "eager"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # This check is now less likely to trigger if model is fully loaded to CPU
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
_inject_lora(model)

# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))


# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# Manually move the model to GPU *after* LoRA injection.
# This ensures the model is on the correct device before Trainer takes over.
if torch.cuda.is_available():
    model.to("cuda")

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

Pyrhon 3.11

In [ ]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.44.0 datasets==2.19.0 accelerate==0.34.0 peft==0.10.0 trl==0.9.6
!pip install bitsandbytes==0.43.1
!pip install numpy==1.26.4 scipy==1.11.4 sentencepiece protobuf==4.25.3



In [ ]:
import os, torch
from typing import Dict, Any, List
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# === Config ===
MODEL_NAME = "google/gemma-2-2b-it"  # Safer for T4. You can switch to "google/gemma-2-2b" after it's stable
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2-lora-dolly"
MAX_LEN      = 512          # start safe on T4. Try 768/1024 later.
EPOCHS       = 1.0
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
RANK         = 16
USE_BF16     = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Tokenizer ===
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# === 4-bit quantization (bnb) ===
if not torch.cuda.is_available():
    raise SystemExit("❌ GPU not found. Switch Colab runtime to GPU (T4/L4/A100).")

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_cfg,
    device_map="auto",
)
base.config.pad_token_id = tok.pad_token_id
base.config.use_cache = False
# Try FA2 if available on your GPU stack
try:
    base.config.attn_implementation = "flash_attention_2"
except Exception:
    pass

# === LoRA via PEFT (no loralib) ===
lora_cfg = LoraConfig(
    r=RANK,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()  # sanity check

# === Dolly formatting ===
def _to_str(x):
    if isinstance(x, list): return " ".join(map(str, x))
    if isinstance(x, str):  return x
    return "" if x is None else str(x)

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()
    prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    if not prompt:
        prompt = "You are a helpful assistant."
    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL expects list[str]

ds = load_dataset(DATASET_NAME)
train_ds = ds["train"]

# === Trainer ===
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,  # memory saver
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tok,
    train_dataset=train_ds,
    args=args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

# quick preview
print("Preview:", format_dolly(train_ds[0])[0][:160].replace("\n"," ⏎ "), "...")

trainer.train()

# === Save adapters only ===
adapter_dir = os.path.join(OUTPUT_DIR, "lora_adapter")
trainer.model.save_pretrained(adapter_dir)
tok.save_pretrained(OUTPUT_DIR)
print("✅ Saved LoRA adapter to:", adapter_dir)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject